<a href="https://colab.research.google.com/github/Mavitu56/SLMs/blob/main/SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workflow Completo: Aprimoramento e Comparação de SLMs

Este notebook é a versão final, completa e funcional do plano para aprimorar e comparar um Small Language Model (SLM) usando Destilação de Conhecimento e Engenharia de Prompt.

### 1. Instalação e Importações Globais

In [ ]:
!pip install torch transformers "datasets==2.19.0" evaluate peft accelerate ipywidgets sentencepiece pandas matplotlib --quiet

In [ ]:
import torch
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from collections import Counter
from tqdm.notebook import tqdm
from datasets import load_dataset, Dataset
from evaluate import load
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.optim import AdamW
import torch.nn.functional as F

### 2. Funções da Fase 0: Configuração do Ambiente

In [ ]:
def configure_task_and_metrics():
    """Carrega o dataset e as métricas para a tarefa de avaliação."""
    print("--- Step 0.1: Configuring Task (SQuAD) and Metrics ---")
    validation_dataset = load_dataset("squad", split="validation")
    squad_metric = load("squad")
    print("Dataset e métricas carregados.")
    return validation_dataset, squad_metric

def load_model(model_id):
    """Carrega um modelo e seu tokenizador a partir de um ID."""
    print(f"\n--- Loading Model: {model_id} ---")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        # Adiciona um pad token se não existir, comum para modelos de geração
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto", torch_dtype=torch.bfloat16
        )
        print("Modelo carregado com sucesso.")
        return model, tokenizer
    except Exception as e:
        print(f"Erro ao carregar o modelo: {e}")
        return None, None

def prepare_datasets(validation_dataset):
    """Prepara todos os subconjuntos de dados necessários."""
    print("\n--- Step 0.4: Preparing Datasets ---")
    squad_train_full = load_dataset("squad", split="train")
    # Usamos um subset menor para o KD Transfer para agilizar o treinamento
    kd_transfer_set = squad_train_full.shuffle(seed=42).select(range(10000))
    icl_cot_set = squad_train_full.select(range(5))
    datasets = {
        "evaluation": validation_dataset,
        "kd_transfer": kd_transfer_set,
        "icl_cot_examples": icl_cot_set
    }
    print("Datasets preparados.")
    return datasets

### 3. Funções da Fase 1: Destilação de Conhecimento (KD)

In [ ]:
class KDConfig:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    TEMPERATURE = 2.0
    LEARNING_RATE = 5e-5
    NUM_EPOCHS = 1 # Reduzido para uma execução mais rápida de exemplo. Aumente para 3 em um treino real.

# --- Técnica 1.1: KD Base (Logit Matching) ---
def compute_distillation_loss(student_logits, teacher_logits, temperature):
    """Calcula a perda de Destilação de Conhecimento (KL Divergence)."""
    soft_teacher_probs = F.softmax(teacher_logits / temperature, dim=-1)
    soft_student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
    distillation_loss = F.kl_div(soft_student_log_probs, soft_teacher_probs, reduction='batchmean')
    return (temperature**2) * distillation_loss

def run_base_kd_training(student_model, teacher_model, student_tokenizer, teacher_tokenizer, kd_dataset, config, output_dir):
    """Executa o loop de treinamento para a Destilação de Conhecimento base."""
    print(f"\n--- Running Base Knowledge Distillation -> saving to {output_dir} ---")
    lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type=TaskType.CAUSAL_LM)
    peft_student_model = get_peft_model(student_model, lora_config)
    optimizer = AdamW(peft_student_model.parameters(), lr=config.LEARNING_RATE)
    teacher_model.eval()

    for epoch in range(config.NUM_EPOCHS):
        print(f"\nEpoch {epoch + 1}/{config.NUM_EPOCHS}")
        peft_student_model.train()
        for batch in tqdm(kd_dataset.shuffle(seed=epoch).select(range(200)), desc=f"Epoch {epoch+1}"):
            prompt = f"Contexto: {batch['context']}\n\nPergunta: {batch['question']}"

            # Tokeniza para o PROFESSOR com o tokenizador do PROFESSOR
            teacher_inputs = teacher_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(config.DEVICE)

            # Tokeniza para o ESTUDANTE com o tokenizador do ESTUDANTE
            student_inputs = student_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(config.DEVICE)

            # Gera os logits de cada modelo com seus respectivos inputs
            with torch.no_grad():
                teacher_logits = teacher_model(**teacher_inputs).logits

            student_logits = peft_student_model(**student_inputs).logits

            # A perda é calculada entre os logits, que devem ter o mesmo tamanho de vocabulário
            # Se os vocabulários forem diferentes, a destilação de logits não é diretamente aplicável
            # e a "Destilação de Explicações" se torna a abordagem correta.
            # Adicionando uma verificação para evitar o erro:
            if student_logits.shape[-1] != teacher_logits.shape[-1]:
                raise ValueError(
                    f"Os tamanhos dos vocabulários do estudante ({student_logits.shape[-1]}) e do professor ({teacher_logits.shape[-1]}) são diferentes. "
                    "A destilação de logits direta não é possível. Use a Destilação de Explicações."
                )

            loss = compute_distillation_loss(student_logits, teacher_logits, config.TEMPERATURE)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    peft_student_model.save_pretrained(output_dir)
    student_tokenizer.save_pretrained(output_dir)
    print(f"Modelo destilado salvo em {output_dir}")

# --- Técnica 1.2: Destilação de Explicações ---
def generate_explanation_dataset(teacher_model, tokenizer, transfer_dataset):
    """Usa o modelo professor para gerar um dataset com raciocínios."""
    print("\n--- Generating Explanation-Augmented Dataset ---")
    prompt_template = "A partir do contexto, responda à pergunta. Explique seu raciocínio passo a passo e termine com a resposta final. Formato: [RACIOCÍNIO] ... <sep> [RESPOSTA] ...\n\nContexto: {context}\nPergunta: {question}"
    new_data = {"text": []}
    for example in tqdm(transfer_dataset.select(range(200)), desc="Generating Explanations"):
        full_prompt = prompt_template.format(context=example['context'], question=example['question'])
        inputs = tokenizer(full_prompt, return_tensors="pt").to(KDConfig.DEVICE)
        outputs = teacher_model.generate(**inputs, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        new_data["text"].append(generated_text)
    return Dataset.from_dict(new_data)

def run_explanation_kd_training(student_model, tokenizer, explanation_dataset, config, output_dir):
    """Treina o estudante para gerar as explicações do professor."""
    print(f"\n--- Running Explanation Distillation -> saving to {output_dir} ---")
    lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type=TaskType.CAUSAL_LM)
    peft_student_model = get_peft_model(student_model, lora_config)
    training_args = TrainingArguments(output_dir="./results/explanation_training", num_train_epochs=config.NUM_EPOCHS, learning_rate=config.LEARNING_RATE, report_to="none")
    trainer = Trainer(model=peft_student_model, args=training_args, train_dataset=explanation_dataset, tokenizer=tokenizer, data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]), 'attention_mask': torch.stack([f['attention_mask'] for f in data]), 'labels': torch.stack([f['input_ids'] for f in data])})
    trainer.train()
    peft_student_model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Modelo de explicação destilada salvo em {output_dir}")

### 4. Funções da Fase 2: Avaliação com Engenharia de Prompt

In [ ]:
def run_zero_shot_baseline(model, tokenizer, evaluation_dataset, metrics_calculator):
    """Executa a avaliação de baseline (Zero-Shot Simples)."""
    prompt = "Contexto: {context}\n\nPergunta: {question}\n\nResposta:"
    return run_generic_evaluation(model, tokenizer, evaluation_dataset, metrics_calculator, prompt, "Zero-Shot Baseline")

def run_few_shot_icl(model, tokenizer, evaluation_dataset, icl_examples, num_shots, metrics_calculator):
    """Executa a avaliação com In-Context Learning (Few-Shot)."""
    prefix = "".join([f"Contexto: {ex['context']}\nPergunta: {ex['question']}\nResposta: {ex['answers']['text'][0]}\n\n---\n\n" for ex in icl_examples.select(range(num_shots))])
    prompt = prefix + "Contexto: {context}\n\nPergunta: {question}\n\nResposta:"
    return run_generic_evaluation(model, tokenizer, evaluation_dataset, metrics_calculator, prompt, f"{num_shots}-Shot ICL")

def run_zero_shot_cot(model, tokenizer, evaluation_dataset, metrics_calculator):
    """Executa a avaliação com Zero-Shot Chain-of-Thought."""
    prompt = "Contexto: {context}\n\nPergunta: {question}\n\nPense passo a passo. A resposta final é:"
    return run_generic_evaluation(model, tokenizer, evaluation_dataset, metrics_calculator, prompt, "Zero-Shot CoT")

def run_self_consistency_cot(model, tokenizer, evaluation_dataset, metrics_calculator, num_paths=5):
    """Executa a avaliação com Auto-Consistência sobre o Zero-Shot CoT."""
    print(f"\n--- Running Self-Consistency ({num_paths} paths) ---")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    prompt_template = "Contexto: {context}\n\nPergunta: {question}\n\nPense passo a passo. A resposta final é:"

    predictions, references = [], []
    for example in tqdm(evaluation_dataset.select(range(20)), desc="Self-Consistency Eval"):
        prompt = prompt_template.format(context=example['context'], question=example['question'])
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        path_answers = []
        for _ in range(num_paths):
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=150, do_sample=True, temperature=0.7, top_k=50, pad_token_id=tokenizer.eos_token_id)
            completion = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):].strip()
            match = re.search(r'A resposta final é:\s*(.*)', completion, re.IGNORECASE)
            if match: path_answers.append(match.group(1).strip())

        final_prediction = Counter(path_answers).most_common(1)[0][0] if path_answers else ""
        predictions.append({'id': example['id'], 'prediction_text': final_prediction})
        references.append({'id': example['id'], 'answers': example['answers']})
    return metrics_calculator.compute(predictions=predictions, references=references)

def run_generic_evaluation(model, tokenizer, eval_dataset, metrics_calculator, prompt_template, strategy_name, eval_subset_size=50):
    """Função genérica que executa a avaliação para a maioria das estratégias de prompt."""
    print(f"\n--- Running Evaluation: {strategy_name} ---")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    predictions, references = [], []

    for example in tqdm(eval_dataset.select(range(eval_subset_size)), desc=f"Evaluating {strategy_name}"):
        prompt = prompt_template.format(context=example['context'], question=example['question'])
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1536, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=60, pad_token_id=tokenizer.eos_token_id)

        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predicted_answer = full_text[len(prompt):].strip()

        if "A resposta final é:" in prompt:
            match = re.search(r'A resposta final é:\s*(.*)', predicted_answer, re.IGNORECASE)
            predicted_answer = match.group(1).strip() if match else predicted_answer.split('\n')[-1].strip()

        predictions.append({'id': example['id'], 'prediction_text': predicted_answer})
        references.append({'id': example['id'], 'answers': example['answers']})

    return metrics_calculator.compute(predictions=predictions, references=references)

### 5. Execução do Pipeline e Geração de Relatório (Fase 3)

In [ ]:
from peft import PeftModel

def run_real_workflow(run_training=False, eval_subset_size=50):
    """
    Orquestra o pipeline completo e REAL: setup, treinamento (opcional), avaliação e relatório.

    Args:
        run_training (bool): Se True, executa a demorada fase de treinamento dos modelos KD.
                             Execute como True uma vez para gerar os modelos. Depois, use False.
        eval_subset_size (int): Número de exemplos do dataset de validação para usar na avaliação.
                                Aumente para uma avaliação mais robusta.
    """
    # --- FASE 0: SETUP ---
    print("--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---")
    validation_data, metrics_calc = configure_task_and_metrics()
    prepared_data = prepare_datasets(validation_data)
    config = KDConfig()

    # --- FASE 1: TREINAMENTO (Opcional) ---
    if run_training:
        print("\n--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---")
        # Carrega os modelos base necessários para todos os treinamentos
        student_model_base, student_tokenizer = load_model("google/gemma-2b")
        teacher_model_ext, teacher_tokenizer = load_model("mistralai/Mistral-7B-Instruct-v0.2")

        # 1. Treina o modelo de KD Base
        run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, tokenizer=tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_base_model"
        )

        # 2. Treina o modelo de Auto-Destilação (requer um professor "especialista")
        print("\n--- Treinando professor para Auto-Destilação ---")
        # A forma mais simples de criar um professor especialista é com um fine-tuning padrão.
        # Aqui, vamos simular isso treinando o próprio estudante com KD por uma época.
        specialist_teacher = run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, tokenizer=tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/specialist_teacher_for_self_distillation"
        )
        print("\n--- Executando Auto-Destilação ---")
        run_base_kd_training(
            student_model=student_model_base, teacher_model=specialist_teacher, tokenizer=tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_self_distilled_model"
        )

        # 3. Treina o modelo de Destilação de Explicações
        explanation_dataset = generate_explanation_dataset(teacher_model_ext, tokenizer, prepared_data['kd_transfer'])
        run_explanation_kd_training(
            student_model=student_model_base, tokenizer=tokenizer, explanation_dataset=explanation_dataset,
            config=config, output_dir="./results/explanation_distilled_model"
        )
        print("\n--- FASE DE TREINAMENTO CONCLUÍDA ---")

    # --- FASE 2 & 3: AVALIAÇÃO E ANÁLISE ---
    print("\n--- FASES 2 & 3: EXECUTANDO AVALIAÇÃO COMPLETA E GERANDO RELATÓRIO ---")
    model_definitions = {
        "SLM Base": "google/gemma-2b",
        "KD Base": "./results/kd_base_model",
        "KD Auto-Destilado": "./results/kd_self_distilled_model",
        "KD com Explicações": "./results/explanation_distilled_model"
    }
    prompting_strategies = {
        "Zero-Shot Simples": run_zero_shot_baseline,
        "ICL (k=3)": lambda m, t, d, met: run_few_shot_icl(m, t, d, prepared_data['icl_cot_examples'], 3, met),
        "Zero-Shot CoT": run_zero_shot_cot,
        "Auto-Consistência (n=5)": lambda m, t, d, met: run_self_consistency_cot(m, t, d, met, 5)
    }
    results_df = pd.DataFrame(index=model_definitions.keys(), columns=prompting_strategies.keys())

    # Carrega o modelo base uma vez para aplicar os adaptadores PEFT
    base_model_for_eval, tokenizer = load_model("google/gemma-2b")

    for alias, path in model_definitions.items():
        print(f"\n>>>> AVALIANDO MODELO: {alias} <<<<")
        if alias == "SLM Base":
            model = base_model_for_eval
        else:
            try:
                # Carrega o modelo base com o adaptador LoRA treinado
                model = PeftModel.from_pretrained(base_model_for_eval, path)
                model = model.merge_and_unload() # Opcional: mescla os pesos para acelerar a inferência
            except Exception as e:
                print(f"Não foi possível carregar o modelo treinado de '{path}'. Pulando. Erro: {e}")
                continue

        for strat_name, strat_func in prompting_strategies.items():
            # A função de avaliação genérica agora é chamada por suas wrappers específicas
            results = strat_func(model, tokenizer, prepared_data['evaluation'], metrics_calc)
            results_df.loc[alias, strat_name] = f"{results['f1']:.1f} / {results['exact_match']:.1f}"

    # --- RELATÓRIO FINAL ---
    print("\n\n## Relatório Final de Análise ##")
    numeric_df = results_df.applymap(lambda x: float(x.split('/')[0]) if isinstance(x, str) else 0)

    print("\n--- Matriz de Resultados (F1-Score / Exact Match) ---")
    print(results_df.to_markdown())

    print("\n--- Conclusões da Análise ---")
    avg_kd_performance = numeric_df.drop("SLM Base").mean(axis=1)
    print(f"- Melhor Técnica de KD (Média F1): '{avg_kd_performance.idxmax()}' ({avg_kd_performance.max():.2f})")
    avg_prompt_performance = numeric_df.mean(axis=0)
    print(f"- Melhor Estratégia de Prompting (Média F1): '{avg_prompt_performance.idxmax()}' ({avg_prompt_performance.max():.2f})")
    best_combo = numeric_df.stack().idxmax()
    print(f"- Melhor Sinergia (Modelo + Prompt): '{best_combo[0]}' + '{best_combo[1]}' com F1 de {numeric_df.max().max():.2f}")

    print("\nGerando gráfico de comparação...")
    fig, ax = plt.subplots(figsize=(12, 7))
    numeric_df.plot(kind='bar', ax=ax, title='Comparação de Performance (F1-Score) entre Modelos e Estratégias')
    ax.set_ylabel("F1-Score")
    ax.set_xlabel("Versão do Modelo")
    ax.tick_params(axis='x', rotation=20)
    plt.legend(title='Estratégia de Prompt', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("real_run_full_performance_comparison.png")
    print("Gráfico salvo em 'real_run_full_performance_comparison.png'.")

### 6. Execução do Pipeline (Modo Simulado)

A célula abaixo executa o pipeline em **modo de simulação**. Ela não treina nem avalia os modelos de verdade, mas gera um relatório com dados de exemplo para demonstrar a estrutura de análise. Para uma execução real, você deve primeiro executar as células de treinamento da Fase 1 e depois rodar o pipeline com `simulation_mode=False`.

In [ ]:
# prompt: login hugging face

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
run_full_pipeline(simulation_mode=True)

NameError: name 'run_full_pipeline' is not defined

In [ ]:
from peft import PeftModel

def run_real_workflow(run_training=False, eval_subset_size=50):
    """
    Orquestra o pipeline completo e REAL: setup, treinamento (opcional), avaliação e relatório.

    Args:
        run_training (bool): Se True, executa a demorada fase de treinamento dos modelos KD.
                             Execute como True uma vez para gerar os modelos. Depois, use False.
        eval_subset_size (int): Número de exemplos do dataset de validação para usar na avaliação.
                                Aumente para uma avaliação mais robusta.
    """
    # --- FASE 0: SETUP ---
    print("--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---")
    validation_data, metrics_calc = configure_task_and_metrics()
    prepared_data = prepare_datasets(validation_data)
    config = KDConfig()

    # --- FASE 1: TREINAMENTO (Opcional) ---
    if run_training:
        print("\n--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---")
        # Carrega os modelos base necessários para todos os treinamentos
        student_model_base, student_tokenizer = load_model("google/gemma-2b")
        teacher_model_ext, teacher_tokenizer = load_model("mistralai/Mistral-7B-Instruct-v0.2")
        print(teacher_model_ext)

        # 1. Treina o modelo de KD Base
        run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_base_model"
        )

        # 2. Treina o modelo de Auto-Destilação (requer um professor "especialista")
        print("\n--- Treinando professor para Auto-Destilação ---")
        # A forma mais simples de criar um professor especialista é com um fine-tuning padrão.
        # Aqui, vamos simular isso treinando o próprio estudante com KD por uma época.
        specialist_teacher = run_base_kd_training(
            student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/specialist_teacher_for_self_distillation"
        )
        print("\n--- Executando Auto-Destilação ---")
        run_base_kd_training(
            student_model=student_model_base, teacher_model=specialist_teacher, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
            kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_self_distilled_model"
        )

        # 3. Treina o modelo de Destilação de Explicações
        explanation_dataset = generate_explanation_dataset(teacher_model_ext, teacher_tokenizer, prepared_data['kd_transfer'])
        run_explanation_kd_training(
            student_model=student_model_base, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer, explanation_dataset=explanation_dataset,
            config=config, output_dir="./results/explanation_distilled_model"
        )
        print("\n--- FASE DE TREINAMENTO CONCLUÍDA ---")

    # --- FASE 2 & 3: AVALIAÇÃO E ANÁLISE ---
    print("\n--- FASES 2 & 3: EXECUTANDO AVALIAÇÃO COMPLETA E GERANDO RELATÓRIO ---")
    model_definitions = {
        "SLM Base": "google/gemma-2b",
        "KD Base": "./results/kd_base_model",
        "KD Auto-Destilado": "./results/kd_self_distilled_model",
        "KD com Explicações": "./results/explanation_distilled_model"
    }
    prompting_strategies = {
        "Zero-Shot Simples": run_zero_shot_baseline,
        "ICL (k=3)": lambda m, t, d, met: run_few_shot_icl(m, t, d, prepared_data['icl_cot_examples'], 3, met),
        "Zero-Shot CoT": run_zero_shot_cot,
        "Auto-Consistência (n=5)": lambda m, t, d, met: run_self_consistency_cot(m, t, d, met, 5)
    }
    results_df = pd.DataFrame(index=model_definitions.keys(), columns=prompting_strategies.keys())

    # Carrega o modelo base uma vez para aplicar os adaptadores PEFT
    base_model_for_eval, tokenizer = load_model("google/gemma-2b")

    for alias, path in model_definitions.items():
        print(f"\n>>>> AVALIANDO MODELO: {alias} <<<<")
        if alias == "SLM Base":
            model = base_model_for_eval
        else:
            try:
                # Carrega o modelo base com o adaptador LoRA treinado
                model = PeftModel.from_pretrained(base_model_for_eval, path)
                model = model.merge_and_unload() # Opcional: mescla os pesos para acelerar a inferência
            except Exception as e:
                print(f"Não foi possível carregar o modelo treinado de '{path}'. Pulando. Erro: {e}")
                continue

        for strat_name, strat_func in prompting_strategies.items():
            # A função de avaliação genérica agora é chamada por suas wrappers específicas
            results = strat_func(model, tokenizer, prepared_data['evaluation'], metrics_calc)
            results_df.loc[alias, strat_name] = f"{results['f1']:.1f} / {results['exact_match']:.1f}"

    # --- RELATÓRIO FINAL ---
    print("\n\n## Relatório Final de Análise ##")
    numeric_df = results_df.applymap(lambda x: float(x.split('/')[0]) if isinstance(x, str) else 0)

    print("\n--- Matriz de Resultados (F1-Score / Exact Match) ---")
    print(results_df.to_markdown())

    print("\n--- Conclusões da Análise ---")
    avg_kd_performance = numeric_df.drop("SLM Base").mean(axis=1)
    print(f"- Melhor Técnica de KD (Média F1): '{avg_kd_performance.idxmax()}' ({avg_kd_performance.max():.2f})")
    avg_prompt_performance = numeric_df.mean(axis=0)
    print(f"- Melhor Estratégia de Prompting (Média F1): '{avg_prompt_performance.idxmax()}' ({avg_prompt_performance.max():.2f})")
    best_combo = numeric_df.stack().idxmax()
    print(f"- Melhor Sinergia (Modelo + Prompt): '{best_combo[0]}' + '{best_combo[1]}' com F1 de {numeric_df.max().max():.2f}")

    print("\nGerando gráfico de comparação...")
    fig, ax = plt.subplots(figsize=(12, 7))
    numeric_df.plot(kind='bar', ax=ax, title='Comparação de Performance (F1-Score) entre Modelos e Estratégias')
    ax.set_ylabel("F1-Score")
    ax.set_xlabel("Versão do Modelo")
    ax.tick_params(axis='x', rotation=20)
    plt.legend(title='Estratégia de Prompt', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("real_run_full_performance_comparison.png")
    print("Gráfico salvo em 'real_run_full_performance_comparison.png'.")

In [ ]:
run_real_workflow(True, 10000)

--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---
--- Step 0.1: Configuring Task (SQuAD) and Metrics ---
Dataset e métricas carregados.

--- Step 0.4: Preparing Datasets ---
Datasets preparados.

--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---

--- Loading Model: google/gemma-2b ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo carregado com sucesso.

--- Loading Model: mistralai/Mistral-7B-Instruct-v0.2 ---


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Erro ao carregar o modelo: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

ERRO CRÍTICO: Falha ao carregar um dos modelos base. O pipeline não pode continuar.


In [ ]:
# --- FASE 0: SETUP ---
print("--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---")
validation_data, metrics_calc = configure_task_and_metrics()
prepared_data = prepare_datasets(validation_data)
config = KDConfig()

run_training  = True

--- FASE 0: CONFIGURANDO AMBIENTE E DADOS ---
--- Step 0.1: Configuring Task (SQuAD) and Metrics ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset e métricas carregados.

--- Step 0.4: Preparing Datasets ---
Datasets preparados.


In [ ]:
# --- FASE 1: TREINAMENTO (Opcional) ---

print("\n--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---")
# Carrega os modelos base necessários para todos os treinamentos
student_model_base, student_tokenizer = load_model("google/gemma-2b")
teacher_model_ext, teacher_tokenizer = load_model("mistralai/Mistral-7B-Instruct-v0.2")


--- FASE 1: EXECUTANDO TREINAMENTO DOS MODELOS KD ---

--- Loading Model: google/gemma-2b ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo carregado com sucesso.

--- Loading Model: mistralai/Mistral-7B-Instruct-v0.2 ---


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Modelo carregado com sucesso.


In [ ]:
# 1. Treina o modelo de KD Base
run_base_kd_training(
    student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
    kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_base_model"
)

# 2. Treina o modelo de Auto-Destilação (requer um professor "especialista")
print("\n--- Treinando professor para Auto-Destilação ---")
# A forma mais simples de criar um professor especialista é com um fine-tuning padrão.
# Aqui, vamos simular isso treinando o próprio estudante com KD por uma época.
specialist_teacher = run_base_kd_training(
    student_model=student_model_base, teacher_model=teacher_model_ext, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
    kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/specialist_teacher_for_self_distillation"
)
print("\n--- Executando Auto-Destilação ---")
run_base_kd_training(
    student_model=student_model_base, teacher_model=specialist_teacher, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer,
    kd_dataset=prepared_data['kd_transfer'], config=config, output_dir="./results/kd_self_distilled_model"
)

# 3. Treina o modelo de Destilação de Explicações
explanation_dataset = generate_explanation_dataset(teacher_model_ext, teacher_tokenizer, prepared_data['kd_transfer'])
run_explanation_kd_training(
    student_model=student_model_base, student_tokenizer=student_tokenizer, teacher_tokenizer=teacher_tokenizer, explanation_dataset=explanation_dataset,
    config=config, output_dir="./results/explanation_distilled_model"
)
print("\n--- FASE DE TREINAMENTO CONCLUÍDA ---")


--- Running Base Knowledge Distillation -> saving to ./results/kd_base_model ---

Epoch 1/1


Epoch 1:   0%|          | 0/200 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 367409 has 14.73 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 30.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# --- FASE 2 & 3: AVALIAÇÃO E ANÁLISE ---
print("\n--- FASES 2 & 3: EXECUTANDO AVALIAÇÃO COMPLETA E GERANDO RELATÓRIO ---")
model_definitions = {
    "SLM Base": "google/gemma-2b",
    "KD Base": "./results/kd_base_model",
    "KD Auto-Destilado": "./results/kd_self_distilled_model",
    "KD com Explicações": "./results/explanation_distilled_model"
}
prompting_strategies = {
    "Zero-Shot Simples": run_zero_shot_baseline,
    "ICL (k=3)": lambda m, t, d, met: run_few_shot_icl(m, t, d, prepared_data['icl_cot_examples'], 3, met),
    "Zero-Shot CoT": run_zero_shot_cot,
    "Auto-Consistência (n=5)": lambda m, t, d, met: run_self_consistency_cot(m, t, d, met, 5)
}
results_df = pd.DataFrame(index=model_definitions.keys(), columns=prompting_strategies.keys())

In [ ]:
# Carrega o modelo base uma vez para aplicar os adaptadores PEFT
base_model_for_eval, tokenizer = load_model("google/gemma-2b")

for alias, path in model_definitions.items():
    print(f"\n>>>> AVALIANDO MODELO: {alias} <<<<")
    if alias == "SLM Base":
        model = base_model_for_eval
    else:
        try:
            # Carrega o modelo base com o adaptador LoRA treinado
            model = PeftModel.from_pretrained(base_model_for_eval, path)
            model = model.merge_and_unload() # Opcional: mescla os pesos para acelerar a inferência
        except Exception as e:
            print(f"Não foi possível carregar o modelo treinado de '{path}'. Pulando. Erro: {e}")
            continue

    for strat_name, strat_func in prompting_strategies.items():
        # A função de avaliação genérica agora é chamada por suas wrappers específicas
        results = strat_func(model, tokenizer, prepared_data['evaluation'], metrics_calc)
        results_df.loc[alias, strat_name] = f"{results['f1']:.1f} / {results['exact_match']:.1f}"

# --- RELATÓRIO FINAL ---
print("\n\n## Relatório Final de Análise ##")
numeric_df = results_df.applymap(lambda x: float(x.split('/')[0]) if isinstance(x, str) else 0)

print("\n--- Matriz de Resultados (F1-Score / Exact Match) ---")
print(results_df.to_markdown())

print("\n--- Conclusões da Análise ---")
avg_kd_performance = numeric_df.drop("SLM Base").mean(axis=1)
print(f"- Melhor Técnica de KD (Média F1): '{avg_kd_performance.idxmax()}' ({avg_kd_performance.max():.2f})")
avg_prompt_performance = numeric_df.mean(axis=0)
print(f"- Melhor Estratégia de Prompting (Média F1): '{avg_prompt_performance.idxmax()}' ({avg_prompt_performance.max():.2f})")
best_combo = numeric_df.stack().idxmax()
print(f"- Melhor Sinergia (Modelo + Prompt): '{best_combo[0]}' + '{best_combo[1]}' com F1 de {numeric_df.max().max():.2f}")

print("\nGerando gráfico de comparação...")
fig, ax = plt.subplots(figsize=(12, 7))
numeric_df.plot(kind='bar', ax=ax, title='Comparação de Performance (F1-Score) entre Modelos e Estratégias')
ax.set_ylabel("F1-Score")
ax.set_xlabel("Versão do Modelo")
ax.tick_params(axis='x', rotation=20)
plt.legend(title='Estratégia de Prompt', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("real_run_full_performance_comparison.png")
print("Gráfico salvo em 'real_run_full_performance_comparison.png'.")